In [405]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics
import math
import pprint
import pickle

### Train-Test Spliting of the Data

In [406]:
data = pd.read_csv('../data/teams_normalized.csv')

# separating the dataset into section ready for model fitting
train, test = train_test_split(data, test_size=0.15, train_size=0.85)
x_cols = ['Age', 'ORtg', 'DRtg', 'NRtg', 'Pace', '3PAr_Norm', 'FTr', 'TS%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA', 'OeFG%', 'OTOV%', 'DRB%', 'OFT/FGA']
y_col = 'W'

x_train = train[x_cols]
y_train = train[y_col]
x_test= test[x_cols]
y_test = test[y_col]

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(957, 16) (957,) (169, 16) (169,)


Here we define some utilities for each of the models we will use.

In [407]:
def create_model(model_name, model):
  pipeline = Pipeline(
     steps=[('scaler', StandardScaler()), (model_name, model)]
  )
  pipeline.fit(x_train, y_train)
  print("model trained and created")
  return pipeline

def score_model(model):
  y_pred = model.predict(x_test)
  print("R2 score =", round(metrics.r2_score(y_test, y_pred), 2))
  print("Mean absolute error =", round(metrics.mean_absolute_error(y_test, y_pred), 2)) 
  print("Root mean squared error =", round(math.sqrt(metrics.mean_squared_error(y_test, y_pred)), 2))

def unpack_model(model_name, model):
  estimator = model.named_steps[model_name]
  pprint.pprint(dict(zip(x_cols, estimator.coef_)))
  print("Intercept: ", estimator.intercept_) 

def save_model(model, filename):
  with open("../models/linear_models/" + filename + ".pickle", "wb") as f:
    pickle.dump(model, f)
  print('Model saved as ' + filename + '.pickle')

### Linear Regression
We can now run the linear regression algorithm on the split data.

In [408]:
linear_regression = create_model('lr', linear_model.LinearRegression())

model trained and created


#### Scoring the Linear Regression Model

In [409]:
score_model(linear_regression)

R2 score = 0.93
Mean absolute error = 2.68
Root mean squared error = 3.52


#### Unpacking the Linear Regression Model

In [410]:
unpack_model('lr', linear_regression)

{'3PAr_Norm': 0.07421875,
 'Age': 0.5968437982889084,
 'DRB%': 0.984375,
 'DRtg': 89339808755542.12,
 'FT/FGA': 6.8671875,
 'FTr': -4.384765625,
 'NRtg': 120005441389091.69,
 'OFT/FGA': -0.748046875,
 'ORB%': 3.16162109375,
 'ORtg': -96094061838271.34,
 'OTOV%': 0.890869140625,
 'OeFG%': -1.953125,
 'Pace': 0.15625,
 'TOV%': -2.68359375,
 'TS%': -5.21484375,
 'eFG%': 10.24609375}
Intercept:  40.466749931923616


While this model is quite accurate, some of the coeficients have really high values, which is a symptom of over fitting. The liner regression model is trying to match the results too acurately, that the algorithm is too data specific.

#### Saving the Linear Regression Model

In [411]:
save_model(linear_regression, 'linear_regression')

Model saved as linear_regression.pickle


### Ridge Model
Since the Linear model has some extremely high coeficient values, we can now use the Ridge model to remedy the overfitting.  

In [412]:
ridge = create_model('ridge', linear_model.Ridge(alpha=0.5))

model trained and created


#### Scoring the Ridge Regression Model

In [413]:
score_model(ridge)

R2 score = 0.93
Mean absolute error = 2.67
Root mean squared error = 3.51


#### Unpacking the Ridge Model

In [414]:
unpack_model('ridge', ridge)

{'3PAr_Norm': 0.05944219333025225,
 'Age': 0.484956218560963,
 'DRB%': 0.9335340895921684,
 'DRtg': -3.3629643627058536,
 'FT/FGA': 3.2131745352518255,
 'FTr': -2.0850609997322187,
 'NRtg': 3.4724239757055253,
 'OFT/FGA': -0.7591366272696555,
 'ORB%': 3.032491889493319,
 'ORtg': 1.209889317371871,
 'OTOV%': 0.9013527003689412,
 'OeFG%': -1.9796341567806608,
 'Pace': 0.15174675250012667,
 'TOV%': -2.5627729971907143,
 'TS%': 0.40472475209972725,
 'eFG%': 4.726839637265068}
Intercept:  40.24764890282132


These coefficients are all much more reasonable in magnitude as compared to the prior regression model

#### Saving the Ridge Model

In [415]:
save_model(ridge, 'ridge')

Model saved as ridge.pickle


### The Lasso Model
Since this data has many columns, we might want the ability to wittle down the effective columns and for this we employ the lasso model.

In [416]:
lasso = create_model('lasso', linear_model.Lasso(alpha=0.1))

model trained and created


#### Scoring the Lasso Model

In [417]:
score_model(lasso)

R2 score = 0.93
Mean absolute error = 2.61
Root mean squared error = 3.53


#### Unpacking the Lasso Model

In [418]:
unpack_model('lasso', lasso)

{'3PAr_Norm': -0.0,
 'Age': 0.44667934680468585,
 'DRB%': -0.0,
 'DRtg': -0.0,
 'FT/FGA': 0.0,
 'FTr': 0.0,
 'NRtg': 10.6594989216597,
 'OFT/FGA': -0.15890880564241391,
 'ORB%': 0.05426359454606889,
 'ORtg': 1.0206473301780712,
 'OTOV%': -0.0,
 'OeFG%': -0.32284845847823557,
 'Pace': 0.26199059685570825,
 'TOV%': -0.0,
 'TS%': 0.23762105536031677,
 'eFG%': 0.0}
Intercept:  40.24764890282132


This model has allowed us to zero in some of the more influential factors.

#### Saving the Lasso Model

In [419]:
save_model(lasso, 'lasso')

Model saved as lasso.pickle


### Combining Concepts: the Elastic Model
These models all have their own strengths and weaknesses, so there is also the elastic model that combines the idea of restricting coefficient values, as well as setting as many to zero as possible.

In [420]:
elastic = create_model('elastic', linear_model.ElasticNet(alpha=0.1, l1_ratio=0.7))

model trained and created


#### Scoring the Elastic Model

In [421]:
score_model(elastic)

R2 score = 0.93
Mean absolute error = 2.63
Root mean squared error = 3.55


#### Unpacking the Elastic Model

In [422]:
unpack_model('elastic', elastic)

{'3PAr_Norm': -0.0,
 'Age': 0.49965566619360674,
 'DRB%': 0.0,
 'DRtg': -2.8369465462138983,
 'FT/FGA': 0.06406669515218708,
 'FTr': 0.0,
 'NRtg': 5.794850538650934,
 'OFT/FGA': -0.3187218611047042,
 'ORB%': 0.8565518561237656,
 'ORtg': 2.9933477618755346,
 'OTOV%': 0.09140414540741745,
 'OeFG%': -0.9237900886115363,
 'Pace': 0.1712946975770053,
 'TOV%': -0.7673890921652069,
 'TS%': 1.3898754660418309,
 'eFG%': 0.6089696302397808}
Intercept:  40.24764890282132


We have now reached a middle ground between the sparsity of factors and the interpretable coefficients with this model.

#### Saving the Elastic Model

In [423]:
save_model(elastic, 'elastic')

Model saved as elastic.pickle
